### Calculate cosine similarity

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
text = ["London Paris London","Paris Paris London"]
cv = CountVectorizer()

count_matrix = cv.fit_transform(text)

In [3]:
print(count_matrix)

  (0, 0)	2
  (0, 1)	1
  (1, 0)	1
  (1, 1)	2


In [4]:
print(count_matrix.toarray())

[[2 1]
 [1 2]]


In [5]:
similarity_scores = cosine_similarity(count_matrix)

print(similarity_scores)

[[1.  0.8]
 [0.8 1. ]]


## Builidng  A Movie Recomendation Engine 

## Step 1: Read CSV File

In [6]:
df = pd.read_csv("../DATA/movie_dataset.csv")

In [7]:
df.head(5)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
df.shape

(4803, 24)

In [9]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')


## Step 2: Select Features

In [10]:
features = ['keywords','cast','genres','director']

## Step 3: Create a column in DF which combines all selected features

In [11]:
def combine_features(row):
    try:
        
        return row['genres']+" "+row['keywords']+" "+row['cast']+" "+row['director']
    except:
        print("Error: ", row)

In [12]:
for feature in features:
    df[feature] = df[feature].fillna('') #filling all NaNs with blank string

df["combined_features"] = df.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column


In [13]:
df["combined_features"].head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
Name: combined_features, dtype: object

In [14]:
df["combined_features"][0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron'

## Step 4: Create count matrix form this new combined column

In [15]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"]) 

In [16]:
count_matrix.shape

(4803, 14845)

In [20]:
# helper functions
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]


## Step: 5 compute the cosine similiartiy based on the count matirx

In [21]:
cosine_sim = cosine_similarity(count_matrix)

In [22]:
cosine_sim

array([[1.        , 0.10540926, 0.12038585, ..., 0.        , 0.        ,
        0.        ],
       [0.10540926, 1.        , 0.0761387 , ..., 0.03651484, 0.        ,
        0.        ],
       [0.12038585, 0.0761387 , 1.        , ..., 0.        , 0.11145564,
        0.        ],
       ...,
       [0.        , 0.03651484, 0.        , ..., 1.        , 0.        ,
        0.04264014],
       [0.        , 0.        , 0.11145564, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04264014, 0.        ,
        1.        ]])

## Step: 6 Get Index of this movie from its title

In [30]:
movie_user_likes = "Avtar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it

IndexError: index 0 is out of bounds for axis 0 with size 0

In [24]:
similar_movies

[(0, 0.12038585308576921),
 (1, 0.0761386987626881),
 (2, 0.9999999999999999),
 (3, 0.16357216402190616),
 (4, 0.2085144140570748),
 (5, 0.12510864843424488),
 (6, 0.0),
 (7, 0.19034674690672024),
 (8, 0.08891084489487741),
 (9, 0.17025130615174974),
 (10, 0.12510864843424488),
 (11, 0.3336230624913197),
 (12, 0.08695652173913045),
 (13, 0.08340576562282992),
 (14, 0.15762208124782012),
 (15, 0.16357216402190616),
 (16, 0.14980117725462766),
 (17, 0.08891084489487741),
 (18, 0.03806934938134405),
 (19, 0.08891084489487741),
 (20, 0.08512565307587487),
 (21, 0.08512565307587487),
 (22, 0.047836487323493986),
 (23, 0.18650096164806276),
 (24, 0.07372097807744857),
 (25, 0.0),
 (26, 0.11821656093586509),
 (27, 0.08512565307587487),
 (28, 0.08512565307587487),
 (29, 0.48900964692182586),
 (30, 0.16357216402190616),
 (31, 0.12510864843424488),
 (32, 0.15762208124782012),
 (33, 0.15762208124782012),
 (34, 0.0),
 (35, 0.09100315103865803),
 (36, 0.1276884796138123),
 (37, 0.09325048082403138)

## Get: 7 Get a list of similar movies in descending order of similarity score

In [25]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

In [26]:
sorted_similar_movies

[(29, 0.48900964692182586),
 (11, 0.3336230624913197),
 (354, 0.2553769592276246),
 (183, 0.25021729686848976),
 (1100, 0.25021729686848976),
 (1137, 0.25021729686848976),
 (1999, 0.25021729686848976),
 (2156, 0.25021729686848976),
 (3336, 0.25021729686848976),
 (3373, 0.24573659359149527),
 (782, 0.24077170617153845),
 (3494, 0.24077170617153845),
 (4638, 0.24077170617153845),
 (1177, 0.23995690956687132),
 (3352, 0.23643312187173018),
 (147, 0.2323209277987099),
 (219, 0.22750787759664506),
 (1958, 0.22750787759664506),
 (444, 0.22470176588194152),
 (1192, 0.22470176588194152),
 (2167, 0.22227711223719354),
 (1024, 0.22116293423234573),
 (206, 0.2173913043478261),
 (1013, 0.2173913043478261),
 (3986, 0.2173913043478261),
 (249, 0.21281413268968719),
 (256, 0.21281413268968719),
 (329, 0.21281413268968719),
 (330, 0.21281413268968719),
 (339, 0.21281413268968719),
 (1068, 0.21281413268968719),
 (3348, 0.21281413268968719),
 (4135, 0.21281413268968719),
 (4, 0.2085144140570748),
 (260,

## Step: 8 Print titles of first 50 Moives

In [27]:
i=0
print("Top 50 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>50:
        break

Top 50 similar movies to Spectre are:

Skyfall
Quantum of Solace
The Girl with the Dragon Tattoo
The Hunger Games: Catching Fire
Johnny English Reborn
One for the Money
The Adventurer: The Curse of the Midas Box
Nancy Drew
Diamonds Are Forever
The Other Side of Heaven
The Spirit
Beastmaster 2: Through the Portal of Time
Amidst the Devil's Wings
Sin City
Action Jackson
Die Another Day
Charlie's Angels: Full Throttle
On the Road
Road to Perdition
Spawn
Nerve
Dick Tracy
Clash of the Titans
Child 44
The Wizard of Oz
Insurgent
Allegiant
The Lord of the Rings: The Return of the King
The Lord of the Rings: The Two Towers
The Incredibles
The Hitchhiker's Guide to the Galaxy
Capote
Gangster's Paradise: Jerusalema
John Carter
Ender's Game
Divergent
The Man from U.N.C.L.E.
The Bourne Identity
The Art of War
The Amazing Spider-Man 2
Damnation Alley
White Fang
Das Boot
2 Guns
The Host
The Glimmer Man
Vampires
Heaven is for Real
Teenage Mutant Ninja Turtles: Out of the Shadows
Eragon
Superman II


In [28]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,combined_features
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron,Action Adventure Fantasy Science Fiction cultu...
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski,Adventure Fantasy Action ocean drug abuse exot...
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes,Action Adventure Crime spy based on novel secr...
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan,Action Crime Drama Thriller dc comics crime fi...
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton,Action Adventure Science Fiction based on nove...


## Recomend for outside dataset

In [31]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director',
       'combined_features'],
      dtype='object')

In [32]:
df_final = df[["title", "combined_features"]]

In [33]:
df_final.head()

,title,combined_features
0,Avatar,Action Adventure Fantasy Science Fiction cultu...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drug abuse exot...
2,Spectre,Action Adventure Crime spy based on novel secr...
3,The Dark Knight Rises,Action Crime Drama Thriller dc comics crime fi...
4,John Carter,Action Adventure Science Fiction based on nove...


In [34]:
df_final.to_csv("../DATA/final_movie_dataset.csv")

In [35]:
df_final.columns

Index(['title', 'combined_features'], dtype='object')

In [36]:
df_final.append({"title":"test", "combined_features":"test row added"}, ignore_index=True)

,title,combined_features
0,Avatar,Action Adventure Fantasy Science Fiction cultu...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drug abuse exot...
2,Spectre,Action Adventure Crime spy based on novel secr...
3,The Dark Knight Rises,Action Crime Drama Thriller dc comics crime fi...
4,John Carter,Action Adventure Science Fiction based on nove...
...,...,...
4799,Newlyweds,Comedy Romance Edward Burns Kerry Bish\u00e9 ...
4800,"Signed, Sealed, Delivered",Comedy Drama Romance TV Movie date love at fir...
4801,Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan R...
4802,My Date with Drew,Documentary obsession camcorder crush dream gi...


In [37]:


def recomend_movie(user_input, count=10):
    
    def get_title_from_index(df,index):
        return df[df.index == index]["title"].values[0]
    
    i = 0
    df = pd.read_csv("../DATA/final_movie_dataset.csv")
    cv=CountVectorizer()
    movie_list=[]
    if user_input == " ":
        return movie_list
    else:
        df = df.append({"title":"external", "combined_features":user_input}, ignore_index=True)
        count_matrix = cv.fit_transform(df["combined_features"])
        cosine_sim = cosine_similarity(count_matrix)
        similar_movies = list(enumerate(cosine_sim[4803]))
        sorted_similar_movies = sorted(similar_movies, key= lambda x:x[1], reverse=True)
        for movie in sorted_similar_movies[1:]:
            movie_list.append(get_title_from_index(df, movie[0]))
            i=i+1
            if i > count:
                break
                
        return movie_list

In [38]:
import random 

In [44]:
i = random.randint(1,4000)
print(df_final["combined_features"].iloc[i])

Action Adventure Science Fiction marvel comic sequel superhero based on comic book vision Robert Downey Jr. Chris Hemsworth Mark Ruffalo Chris Evans Scarlett Johansson Joss Whedon


In [45]:
user = "Comedy Drama Comedy prisoners  martial Joss Whedon Thriller arts  game suicide southern usa adultery kung fu biography transport Jason Robards John Bennett Perry William A. Fraker fbi agent death"

In [46]:
recomend_movie(user)

['Me You and Five Bucks',
 'Crying with Laughter',
 "Charlie's Angels",
 'Down & Out With The Dolls',
 'Lone Wolf McQuade',
 'The Grandmaster',
 'Over Her Dead Body',
 'Animal House',
 'The Hudsucker Proxy',
 'My Big Fat Greek Wedding 2',
 'Miss Congeniality 2: Armed and Fabulous']